In [10]:
import numpy as np
import random
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix, random_density_matrix
from qiskit.quantum_info.operators import Operator
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.linalg import sqrtm
from scipy.optimize import minimize

import torch
#tensorflow imports
from tensorflow import keras
from keras import layers, losses, Model
import logging
from functools import reduce
from itertools import product
import time
tf.get_logger().setLevel(logging.ERROR)

# 1.0) Comparing runtimes

In [23]:
# Fidelity
def fidelity(rho1, rho2):
    sqrt_rho1 = sqrtm(rho1)
    F = np.trace(sqrtm(sqrt_rho1 @ rho2 @ sqrt_rho1))
    return np.real(F)**2

def kron_all(mats):
    return reduce(np.kron, mats)

# Map the cholesky parameterised vector to a PSD trace 1 density matrix
def params_to_rho(d, params):
    L = np.zeros((d,d), dtype=complex)
    idx = 0
    # diagonal entries (real, positive)
    for i in range(d):
        L[i, i] = params[idx]
        idx += 1
    # lower-triangular off-diagonals (real + imag)
    for i in range(1, d):
        for j in range(i):
            re = params[idx]; im = params[idx+1]
            L[i, j] = re + 1j * im
            idx += 2
    rho = L @ L.conj().T
    return rho / np.trace(rho)

# Single-qubit unitaries
def build_single_qubit_Us():
    X_cols = [np.array([1, 1])/np.sqrt(2),
              np.array([1,-1])/np.sqrt(2)]
    Y_cols = [np.array([1, 1j])/np.sqrt(2),
              np.array([1,-1j])/np.sqrt(2)] 
    Z_cols = [np.array([1,0]), np.array([0,1])]
    return {'X': np.column_stack(X_cols),
            'Y': np.column_stack(Y_cols),
            'Z': np.column_stack(Z_cols)}

# POVM builder for arbitrary d = 2^n
def build_povm(d):
    n_qubits = int(np.log2(d))
    Us1 = build_single_qubit_Us()
    settings = []
    for bases in product(['X','Y','Z'], repeat=n_qubits):
        U = kron_all([Us1[b] for b in bases])
        settings.append(U)

    # computational-basis projectors
    proj = []
    for m in range(d):
        P = np.zeros((d, d), dtype=complex)
        P[m, m] = 1.0
        proj.append(P)

    # rotated projectors
    E = []
    for U in settings:
        U_dag = U.conj().T
        for P in proj:
            E.append(U @ P @ U_dag)
    return E  

# Negative log-likelihood
def neg_log_likelihood(params, counts, d, E):
    rho = params_to_rho(d, params)
    probs = np.array([np.trace(Ei @ rho).real for Ei in E])
    probs = np.clip(probs, 1e-15, 1.0)
    return -np.sum(counts * np.log(probs))

# MLE with runtime
def MLE_Avg_fidelity_with_runtime(data, X, y, d):
    shots = data['shots_per_basis']
    counts = (X * shots).astype(int) 

    N = X.shape[0]
    rho_est = np.zeros((N, d, d), dtype=complex)
    fidelities = np.zeros(N)
    runtimes = []

    # build POVM once for this d
    E = build_povm(d)

    for i in range(N):
        # initial guess: uniform identity
        init = np.zeros(d*d)
        init[:d] = np.sqrt(1/d)

        start = time.perf_counter()
        res = minimize(
            neg_log_likelihood, init, args=(counts[i], d, E),
            method='L-BFGS-B',
            options={'maxiter': 50}
        )
        end = time.perf_counter()
        
        runtimes.append(end - start)

        # reconstruct density matrix
        rho_est[i] = params_to_rho(d, res.x)
        fidelities[i] = fidelity(rho_est[i], y[i])

    total_runtime = np.sum(runtimes)
    avg_runtime   = np.mean(runtimes)
    return total_runtime, avg_runtime, np.mean(fidelities)




def rho_to_alpha(rho):
    """
    Convert a (d x d) density matrix rho into its "Cholesky parameters" alpha.
    """
    L = np.linalg.cholesky(rho + 1e-14 * np.eye(rho.shape[0]))
    d = rho.shape[0]
    alpha = []
    # diagonal (real, >0)
    for i in range(d):
        alpha.append(np.real(L[i, i]))
    # strictly lower triangle (real + imag)
    for i in range(1, d):
        for j in range(i):
            alpha.append(np.real(L[i, j]))
            alpha.append(np.imag(L[i, j]))
    return np.array(alpha, dtype=np.float32)

def fidelity(rho1, rho2):
    A = sqrtm(rho1)
    return np.real(np.trace(sqrtm(A @ rho2 @ A)))**2


def tf_sqrtm_psd(A):
    """
    Compute principal sqrt of Hermitian PSD A (batch,d,d) via eigendecomposition.
    """
    
    eigvals, eigvecs = tf.linalg.eigh(A)

    eigvals = tf.math.real(eigvals)
    eigvals = tf.clip_by_value(eigvals, 0.0, tf.reduce_max(eigvals))
    sqrtvals = tf.sqrt(eigvals)
    D = tf.cast(tf.linalg.diag(sqrtvals), tf.complex64)
    
    return eigvecs @ D @ tf.linalg.adjoint(eigvecs)

def tf_alpha_to_rho(alpha, d):
    """
    Map real alpha (batch, N_alpha) -> complex density matrices (batch, d, d).
    Enforces positivity via a softplus on the Cholesky diag.
    """
    batch = tf.shape(alpha)[0]
    # split diag vs off-diag
    raw_diag = alpha[:, :d] 
    off_vals  = alpha[:, d:] 

    # start zero L
    L = tf.zeros((batch, d, d), tf.complex64)

    diag_pos = tf.nn.softplus(raw_diag) + 1e-6 
    diag_c   = tf.cast(diag_pos, tf.complex64)
    L = tf.linalg.set_diag(L, diag_c)

    idx = 0
    for i in range(1, d):
        for j in range(i):
            re = off_vals[:, idx]
            im = off_vals[:, idx+1]
            idx += 2
            cij = (tf.cast(re, tf.complex64)
                   + 1j * tf.cast(im, tf.complex64))
            cij = tf.reshape(cij, (batch, 1, 1))

            # mask with a one-hot at (i,j)
            flat = tf.one_hot(i*d + j, d*d, dtype=tf.complex64)
            mask = tf.reshape(flat, (d, d))[None, :, :]

            L = L + cij * mask

    rho = L @ tf.linalg.adjoint(L)  
    tr  = tf.linalg.trace(rho)
    return rho / tr[:, None, None]

def make_fidelity_loss(d):
    def fidelity_loss(alpha_true, alpha_pred):
        rho_t = tf_alpha_to_rho(alpha_true, d)
        rho_p = tf_alpha_to_rho(alpha_pred, d)

        # tiny regularizer to guard numeric issues
        I = tf.eye(d, dtype=tf.complex64)[None, :, :]
        rho_t = rho_t + 1e-8 * I
        rho_p = rho_p + 1e-8 * I

        sqrt_t = tf_sqrtm_psd(rho_t)
        inter  = sqrt_t @ (rho_p @ sqrt_t)
        s_mat  = tf_sqrtm_psd(inter)

        tr_s = tf.linalg.trace(s_mat)
        F = tf.abs(tr_s)**2
        return tf.reduce_mean(1.0 - F)
    return fidelity_loss

def make_hybrid_loss(d, lam=0.8):
    """
    Hybrid loss = lam * MSE + (1 - lam) * (1 - fidelity).
    """
    fid_loss_fn = make_fidelity_loss(d)

    def hybrid_loss(alpha_true, alpha_pred):
        # MSE on the Cholesky parameters
        mse = tf.reduce_mean(tf.square(alpha_true - alpha_pred))

        # fidelity loss already = mean(1 - F)
        phys = fid_loss_fn(alpha_true, alpha_pred)

        return lam * mse + (1.0 - lam) * phys

    return hybrid_loss

def alpha_to_rho_batch(d, alpha):
    """Convert batch of alpha vectors to density matrices using Cholesky."""
    N = alpha.shape[0]
    rho = np.zeros((N, d, d), dtype=np.complex64)
    for i in range(N):
        a = alpha[i]
        L = np.zeros((d, d), dtype=np.complex64)
        idx = 0
        for j in range(d):
            L[j, j] = a[idx]
            idx += 1
        for j in range(1, d):
            for k in range(j):
                re = a[idx]
                im = a[idx + 1]
                L[j, k] = re + 1j * im
                idx += 2
        rho_i = L @ L.conj().T
        rho[i] = rho_i / np.trace(rho_i)
    return rho

def train_NN_fidelity(X, y, lam=0.8, epochs=30):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=2)

    alphas = np.stack([rho_to_alpha(y[i]) for i in range(len(y))], axis=0)
    d = y_train.shape[1]

    alphas_train = np.stack([rho_to_alpha(rho) for rho in y_train], axis=0)
    N_alpha = alphas_train.shape[1]

    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(N_alpha),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=make_hybrid_loss(d, lam=lam),
    )

    # fit
    history = model.fit(
        X_train, alphas_train,
        validation_split=0.1,
        epochs=epochs,
        batch_size=64,
    )

    alpha_pred = model.predict(X_test)      
    rho_pred = alpha_to_rho_batch(d, alpha_pred)  

    # Compute fidelities
    fidelities = np.array([
        fidelity(rho_pred[i], y_test[i])
        for i in range(len(rho_pred))
    ])

    # Summary
    mean_fid = np.mean(fidelities)
    std_fid = np.std(fidelities)
    return(mean_fid, std_fid)


def make_hybrid_loss(d, lam):
    lam = tf.convert_to_tensor(lam, dtype=tf.float32) 
    fid_loss_fn = make_fidelity_loss(d)
    def hybrid_loss(alpha_true, alpha_pred):
        mse = tf.reduce_mean(tf.square(alpha_true - alpha_pred))
        phys = fid_loss_fn(alpha_true, alpha_pred)
        return lam * mse + (1.0 - lam) * phys

    return hybrid_loss

class LambdaAnnealer(tf.keras.callbacks.Callback):
    def __init__(self, lam_var, lam_start, lam_end, epochs):
        super().__init__()
        self.lam_var = lam_var
        self.lam_start = float(lam_start)
        self.lam_end = float(lam_end)
        self.epochs = int(epochs)

    def on_train_begin(self, logs=None):
        self.lam_var.assign(self.lam_start)

    def on_epoch_begin(self, epoch, logs=None):
        t = epoch / max(1, self.epochs - 1)
        lam = self.lam_start + t * (self.lam_end - self.lam_start)
        self.lam_var.assign(lam)
        if logs is not None:
            logs['lambda'] = float(lam)

def train_anneal_NN(X, y, mle_idx, lam_start=0.95, lam_end=0.10, epochs=30):

    X_test = X[mle_idx]; y_test = y[mle_idx]
    X_train = np.delete(X, mle_idx, axis=0)
    y_train = np.delete(y, mle_idx, axis=0)

    d = y_train.shape[1]
    alphas_train = np.stack([rho_to_alpha(rho) for rho in y_train], axis=0)
    N_alpha = alphas_train.shape[1]

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.GaussianNoise(0.02),
        tf.keras.layers.LayerNormalization(),

        tf.keras.layers.Dense(512, activation="gelu"),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(512, activation="gelu"),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(512, activation="gelu"),
        tf.keras.layers.LayerNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(256, activation="gelu"),
        tf.keras.layers.LayerNormalization(),

        tf.keras.layers.Dense(N_alpha),
    ])

    lam_var = tf.Variable(lam_start, trainable=False, dtype=tf.float32, name="lambda_weight")

    opt = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=1e-4, clipnorm=1.0)

    model.compile(
        optimizer=opt,
        loss=make_hybrid_loss(d, lam=lam_var),
    )

    class ValFidelity(tf.keras.callbacks.Callback):
        def __init__(self, Xv, yv, every=1, k=512):
            super().__init__()
            self.idx = np.random.RandomState(42).choice(len(Xv), size=min(k, len(Xv)), replace=False)
            self.Xv = Xv[self.idx]
            self.yv = yv[self.idx]

        def on_epoch_end(self, epoch, logs=None):
            alpha_pred = self.model(self.Xv, training=False).numpy()
            rho_pred = alpha_to_rho_batch(d, alpha_pred)
            F = np.mean([fidelity(rho_pred[i], self.yv[i]) for i in range(len(rho_pred))])
            if logs is not None:
                logs['val_fidelity'] = float(F)
            print(f"Epoch {epoch+1}: val_fidelity={F:.4f}")

    fid_cb = ValFidelity(X_test, y_test, every=1, k=512)

    cb = [
        LambdaAnnealer(lam_var, lam_start=lam_start, lam_end=lam_end, epochs=epochs),
        fid_cb,
    ]

    history = model.fit(
        X_train, alphas_train,
        validation_split=0.1,
        epochs=epochs,
        batch_size=128,
        callbacks=cb,
        verbose=1,
        shuffle=True,
    )

    alpha_pred = model.predict(X_test, verbose=0)
    rho_pred = alpha_to_rho_batch(d, alpha_pred)

    fidelities = np.array([fidelity(rho_pred[i], y_test[i]) for i in range(len(rho_pred))])
    return np.mean(fidelities), np.std(fidelities)


In [13]:
#load the data

#2 qubit
data_2q = np.load('../datasets/2_qubit_shots/2q_1k_.npz')
X_2q = data_2q['counts'] / data_2q['shots_per_basis']; y_2q = data_2q['states']

#3 qubit
data_3q = np.load('../datasets/3_qubit_shots/3q_1000.npz')
X_3q = data_3q['counts'] / data_3q['shots_per_basis']; y_3q = data_3q['states']


In [59]:
rng = np.random.default_rng(42)
mle_indices = rng.choice(100_000, size=1, replace=False)
results_2q = MLE_Avg_fidelity_with_runtime(data_2q, X_2q[mle_indices], y_2q[mle_indices], d=4)



In [34]:
results_3q = MLE_Avg_fidelity_with_runtime(data_3q, X_3q[mle_indices], y_3q[mle_indices], d=8)
results_3q[0]*1000

1690.9488750388846

In [33]:
results_2q[0]*1000

98.70349999982864

In [28]:
results_2q[0]*1000

62491.66885833256

In [60]:
mle_indices

array([8925])

In [50]:
X_2q[mle_indices].shape

(36,)

In [65]:
def train_NN_fidelity_runtime_at_index(
    X, y, test_index, lam=0.8, epochs=30, batch_size=64, validation_split=0.1
):
    """
    Train on all samples except `test_index`, then evaluate inference & fidelity on that one sample.

    Returns:
      {
        "training_time_s": float,
        "inference_time_single_s": float,
        "fidelity_single": float,
        "n_params_total": int,
        "n_params_trainable": int,
        "d": int,
        "test_index_used": int
      }

    Expects helper functions in scope:
      - rho_to_alpha(rho)
      - make_hybrid_loss(d, lam)
      - alpha_to_rho_batch(d, A)
      - fidelity(rho_pred, rho_gt)
    """
    N = len(y)
    if not (0 <= test_index < N):
        raise ValueError(f"test_index {test_index} is out of range [0, {N-1}].")

    d = y.shape[1]

    mask = np.ones(N, dtype=bool)
    mask[test_index] = False
    X_train, y_train = X[mask], y[mask]
    x_one = X[test_index:test_index+1]
    y_one = y[test_index]

    alphas_train = np.stack([rho_to_alpha(rho) for rho in y_train], axis=0)
    N_alpha = alphas_train.shape[1]

    # Model
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dense(N_alpha),
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss=make_hybrid_loss(d, lam=lam),
    )

    t0 = time.perf_counter()
    _ = model.fit(
        X_train, alphas_train,
        validation_split=validation_split,
        epochs=epochs,
        batch_size=batch_size,
        shuffle=True
    )
    training_time_s = time.perf_counter() - t0

    _ = model.predict(x_one, verbose=0)

    t1 = time.perf_counter()
    alpha_pred_one = model.predict(x_one, verbose=0)  # (1, N_alpha)
    inference_time_single_s = time.perf_counter() - t1

    rho_pred_one = alpha_to_rho_batch(d, alpha_pred_one)[0]
    fidelity_single = float(fidelity(rho_pred_one, y_one))

    n_params_total = int(model.count_params())
    n_params_trainable = int(np.sum([np.prod(v.shape) for v in model.trainable_weights]))

    return {
        "training_time_s": training_time_s,
        "inference_time_single_s": inference_time_single_s,
        "fidelity_single": fidelity_single,
        "n_params_total": n_params_total,
        "n_params_trainable": n_params_trainable,
        "d": int(d),
        "test_index_used": int(test_index),
    }

In [63]:
train_NN_fidelity_runtime_at_index(X_2q, y_2q, test_index = 8925)

Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.1134 - val_loss: 0.0106
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0169 - val_loss: 0.0084
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0139 - val_loss: 0.0080
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0127 - val_loss: 0.0073
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0119 - val_loss: 0.0069
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0113 - val_loss: 0.0067
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0110 - val_loss: 0.0064
Epoch 8/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0109 - val_loss: 0.0062
Epoch 9/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0107 - val_loss: 0.0067
Epoch 10/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0105 - val_loss: 0.0062
Epoch 11/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0103 - val_loss: 0.0062
Epoch 12/30
1407/1407 ━━━━━━━━

{'training_time_s': 61.21716291597113,
 'inference_time_single_s': 0.023326750029809773,
 'fidelity_single': 0.9871515984509145,
 'n_params_total': 23968,
 'n_params_trainable': 23640,
 'd': 4,
 'test_index_used': 8925}

In [66]:
train_NN_fidelity_runtime_at_index(X_3q, y_3q, test_index = 8925)

Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - loss: 0.1094 - val_loss: 0.0096
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0112 - val_loss: 0.0065
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0085 - val_loss: 0.0059
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0075 - val_loss: 0.0056
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0069 - val_loss: 0.0055
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0066 - val_loss: 0.0053
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0062 - val_loss: 0.0051
Epoch 8/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0058 - val_loss: 0.0049
Epoch 9/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0055 - val_loss: 0.0048
Epoch 10/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 0.0053 - val_loss: 0.0047
Epoch 11/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - loss: 0.0052 - val_loss: 0.0047
Epoch 12/30
1407/1407 ━━━━━━

{'training_time_s': 277.1455034579849,
 'inference_time_single_s': 0.024929082952439785,
 'fidelity_single': 0.9834591053720451,
 'n_params_total': 409504,
 'n_params_trainable': 408048,
 'd': 8,
 'test_index_used': 8925}